### 추천 시스템

- 하나의 콘텐츠를 선택했을 때 선택된 콘텐츠와 연관된 추천 콘텐츠가 얼마나 사용자의 관심을 끌고 개인에게 맞춘 콘텐츠를 추천했는지는 사용자가 해당 사이트를 더 강하게 신뢰할 수 있도록 하는 중요한 요소입니다.
- 더 많은 데이터가 추천 시스템에 축적되면서 추천이 더욱 정확해지고 다양한 결과를 얻을 수 있는 좋은 선순환 시스템을 구축하는 것이 중요합니다.

- 추천 시스템의 유형
    - 콘텐츠 기반 필터링
    - 협업 필터링
        - 최근접 이웃 협업 필터링
        - 잠재 요인 협업 필터링


#### 콘텐츠 기반 필터링

In [16]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movie_df = pd.read_csv('tmdb_5000_movies.csv')
print(movie_df.shape)
movie_df.head(1)


(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [17]:
movie_df = movie_df[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [18]:
pd.set_option('max_colwidth',100)

In [19]:
movie_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."


In [20]:
from ast import literal_eval
movie_df['genres'] = movie_df['genres'].apply(literal_eval)
movie_df['keywords'] = movie_df['keywords'].apply(literal_eval)

In [21]:
movie_df['genres'] = movie_df['genres'].apply(lambda x: [y['name'] for y in x])
movie_df['keywords'] = movie_df['keywords'].apply(lambda x : [y['name'] for y in x ])
movie_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


#### 장르 콘텐츠 유사도 측정

- 영화별 장르 유사도를 측정하기 위해서는 장르를 문자열로 변경한 뒤 피처 벡터화한 후 행렬 데이터 값을 코사인 유사도로 비교하는 것입니다.

- genres 칼럼을 기반으로 하는 콘텐츠 기반 필터링 과정
    1. genres 칼럼을 문자열로 변환 후 Count 피처 벡터화 변환
    2. 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교, 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성
    3. 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화 추천

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

movie_df['gernes_literal'] = movie_df['genres'].apply(lambda x:(' ').join(x))
count_vect = CountVectorizer(min_df=0,ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movie_df['gernes_literal'])
print(genre_mat.shape)

(4803, 276)


In [23]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]
